In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Dropout, Flatten, Dense, Input
from keras import applications
from keras.layers import AveragePooling2D, GlobalAveragePooling2D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping, CSVLogger
from keras.applications.mobilenet import MobileNet
from keras.layers import Conv2D, Convolution2D, MaxPooling2D, ZeroPadding2D, BatchNormalization, Activation
from keras.optimizers import RMSprop
from keras import backend as K
import numpy as np
import pandas as pd
from keras import layers
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from keras.models import load_model

Using TensorFlow backend.


In [2]:
#parameters
classes = 12
image_size = 224
batch_size= 32
train_total = 3783
validation_total = 967

In [3]:
model_mobilenet = MobileNet(include_top=False, weights = 'imagenet',input_shape=(image_size,image_size,3), alpha=0.5)

In [4]:
x = model_mobilenet.output
x = Dropout(0.5)(x)
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(classes, activation='softmax')(x)

In [5]:
model = Model(inputs=model_mobilenet.input, outputs=x)

In [6]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 16)      432       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 16)      64        
_________________________________________________________________
conv1_relu (Activation)      (None, 112, 112, 16)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 16)      144       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 16)      64        
_________________________________________________________________
conv_dw_1_relu (Activation)  (None, 112, 112, 16)      0         
__________

In [7]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range = 0.2,
                                   rotation_range = 20,
                                   height_shift_range=0.2,
                                   width_shift_range=0.2,
                                   zoom_range=0.2,
                                   fill_mode='reflect',
                                   horizontal_flip=True,
                                   vertical_flip=True)
validation_datagen = ImageDataGenerator(rescale = 1./255)

In [8]:
train_generator = train_datagen.flow_from_directory(
                    'train_second_split/',
                    target_size=(image_size,image_size),
                    batch_size=batch_size,
                    class_mode='categorical',
                    shuffle=True
                    )
validation_generator = validation_datagen.flow_from_directory(
                        'validation_second_split/',
                        target_size=(image_size,image_size),
                        batch_size=batch_size,
                        class_mode='categorical')

Found 3783 images belonging to 12 classes.
Found 967 images belonging to 12 classes.


In [9]:
model.compile(loss='categorical_crossentropy',
             optimizer=RMSprop(lr=1e-4),
             metrics=['accuracy'])

In [10]:
model.load_weights('mobilenet_from_scratch_second_split.hdf5')

In [11]:
model.evaluate_generator(validation_generator, int(np.ceil(validation_total/batch_size)))

[0.069503416318767594, 0.97518097207859356]

In [12]:
# MODEL SMALL FOR BINARY CLASSIFICATION FOR CLASS 0 AND CLASS 6
#V2 - weights from previous model_1, fixed bottom, only fine tuning last layers

In [13]:
#parameters
classes_2 = 2
image_size_2 = 224
batch_size_2 = 32
train_total_2 = 733
validation_total_2 = 184

In [14]:
model_mobilenet_2 = MobileNet(include_top=False, weights = None,input_shape=(image_size_2,image_size_2,3), alpha=0.5)

In [15]:
x = model_mobilenet_2.output
x = Dropout(0.5)(x)
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(classes_2, activation='softmax')(x)

In [16]:
model_2 = Model(inputs=model_mobilenet_2.input, outputs=x)

In [17]:
model_2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 16)      432       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 16)      64        
_________________________________________________________________
conv1_relu (Activation)      (None, 112, 112, 16)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 16)      144       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 16)      64        
_________________________________________________________________
conv_dw_1_relu (Activation)  (None, 112, 112, 16)      0         
__________

In [18]:
train_datagen_2 = ImageDataGenerator(rescale=1./255,
                                   shear_range = 0.2,
                                   rotation_range = 90,
                                   height_shift_range=0.2,
                                   width_shift_range=0.2,
                                   zoom_range=0.2,
                                   fill_mode='reflect',
                                   horizontal_flip=True,
                                   vertical_flip=True)
validation_datagen_2 = ImageDataGenerator(rescale = 1./255)

In [19]:
train_generator_2 = train_datagen_2.flow_from_directory(
                    'train_2_second_split',
                    target_size=(image_size_2,image_size_2),
                    batch_size=batch_size_2,
                    class_mode='categorical',
                    shuffle=True
                    )
validation_generator_2 = validation_datagen_2.flow_from_directory(
                        'validation_2_second_split',
                        target_size=(image_size_2,image_size_2),
                        batch_size=batch_size_2,
                        class_mode='categorical')

Found 733 images belonging to 2 classes.
Found 184 images belonging to 2 classes.


In [20]:
model_2.compile(loss='categorical_crossentropy',
             optimizer=RMSprop(lr=1e-4),
             metrics=['accuracy'])

In [21]:
callbacks = [ModelCheckpoint(filepath='11_mobilenet_from_scratch_model_2_v3_second_split.hdf5', verbose=1, save_best_only=True, save_weights_only=True),
ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, verbose=1),
EarlyStopping(monitor='val_loss', patience=10, verbose=1),
CSVLogger('./11-metrics_model_2_v3_second_split.csv')]

In [22]:
# loading weights from model_1 to model_2


In [23]:
for i in range(len(model.layers)-2):
    layer_1 = model.layers[i]
    layer_2 = model_2.layers[i]
    layer_2.set_weights(layer_1.get_weights())

In [24]:
x = np.zeros((256,2))
x.shape

(256, 2)

In [25]:
y = np.zeros(2)

In [26]:
y[0]=model.layers[-1].get_weights()[1][0]
y[1]=model.layers[-1].get_weights()[1][6]

In [27]:
x[:,0]=model.layers[-1].get_weights()[0][:,0]
x[:,1]=model.layers[-1].get_weights()[0][:,6]

In [28]:
model_2.layers[-1].set_weights([x,y])

In [29]:
validation_generator_2 = validation_datagen_2.flow_from_directory(
                        'validation_2_second_split',
                        target_size=(image_size_2,image_size_2),
                        batch_size=batch_size_2,
                        class_mode='categorical')

Found 184 images belonging to 2 classes.


In [56]:
history_2 = model_2.fit_generator(
                    train_generator_2,
                    steps_per_epoch = int(np.ceil(train_total_2/batch_size_2)),
                    epochs=100,
                    validation_data=validation_generator_2,
                    validation_steps= int(np.ceil(validation_total_2/batch_size_2)),
                    verbose=2,
                    callbacks=callbacks)

Epoch 1/100
Epoch 00000: val_loss improved from inf to 0.21699, saving model to 11_mobilenet_from_scratch_model_2_v3_second_split.hdf5
22s - loss: 0.4199 - acc: 0.8282 - val_loss: 0.2170 - val_acc: 0.9130
Epoch 2/100
Epoch 00001: val_loss improved from 0.21699 to 0.18428, saving model to 11_mobilenet_from_scratch_model_2_v3_second_split.hdf5
19s - loss: 0.4289 - acc: 0.8307 - val_loss: 0.1843 - val_acc: 0.9185
Epoch 3/100
Epoch 00002: val_loss improved from 0.18428 to 0.17060, saving model to 11_mobilenet_from_scratch_model_2_v3_second_split.hdf5
17s - loss: 0.3712 - acc: 0.8500 - val_loss: 0.1706 - val_acc: 0.9402
Epoch 4/100
Epoch 00003: val_loss did not improve
17s - loss: 0.2957 - acc: 0.8831 - val_loss: 0.2088 - val_acc: 0.9185
Epoch 5/100
Epoch 00004: val_loss did not improve
17s - loss: 0.3284 - acc: 0.8652 - val_loss: 0.2094 - val_acc: 0.9022
Epoch 6/100
Epoch 00005: val_loss did not improve
19s - loss: 0.3084 - acc: 0.8719 - val_loss: 0.2641 - val_acc: 0.8750
Epoch 7/100
Epoch

KeyboardInterrupt: 

In [30]:
model_2.load_weights('11_mobilenet_from_scratch_model_2_v3_second_split.hdf5')

In [31]:
model_2.evaluate_generator(validation_generator_2, int(np.ceil(validation_total_2/batch_size_2)))

[0.17926568764707315, 0.93478260610414587]

In [32]:
model_mobilenet_3 = MobileNet(include_top=False, weights = 'imagenet',input_shape=(224,224,3), alpha=0.5)
x = model_mobilenet_3.output
x = Dropout(0.5)(x)
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(2, activation='softmax')(x)
model_3 = Model(inputs=model_mobilenet_3.input, outputs=x)

In [38]:
model_3.load_weights('10_mobilenet_from_scratch_save_best_model_2_v3.hdf5')

In [39]:
model_3.compile(loss='categorical_crossentropy',
             optimizer=RMSprop(lr=1e-4),
             metrics=['accuracy'])

In [40]:
validation_generator_3 = validation_datagen_2.flow_from_directory(
                        'validation_2',
                        target_size=(image_size_2,image_size_2),
                        batch_size=batch_size_2,
                        class_mode='categorical')
model_3.evaluate_generator(validation_generator_3, int(np.ceil(178/batch_size_2)))

Found 178 images belonging to 2 classes.


[0.65319951866449932, 0.882022469900967]

In [33]:
# averaging for model 2 
for i in range(len(model_2.layers)):
    res_weights = [(model_2.layers[i].get_weights()[j]+model_3.layers[i].get_weights()[j])/2 for j in range(len(model_2.layers[i].get_weights()))]
    model_2.layers[i].set_weights(res_weights)

In [34]:
validation_generator_2 = validation_datagen_2.flow_from_directory(
                        'validation_2_second_split',
                        target_size=(image_size_2,image_size_2),
                        batch_size=batch_size_2,
                        class_mode='categorical',
                        shuffle=False)

Found 184 images belonging to 2 classes.


In [35]:
predictions_valid_2 = model_2.predict_generator(validation_generator_2,int(np.ceil(validation_total_2/batch_size_2)))
predictions_valid_2 = np.argmax(predictions_valid_2, axis=1)


In [36]:
len(predictions_valid_2)

184

In [37]:
true_positive_2 = np.zeros(2)
false_positive_2 = np.zeros(2)
true_negative_2 = np.zeros(2)
false_negative_2 = np.zeros(2)
for i in range(len(predictions_valid_2)):
    if(predictions_valid_2[i]==validation_generator_2.classes[i]):
        true_positive_2[predictions_valid_2[i]] = true_positive_2[predictions_valid_2[i]] + 1
    else:
        false_positive_2[predictions_valid_2[i]] = false_positive_2[predictions_valid_2[i]] + 1
        false_negative_2[validation_generator_2.classes[i]] = false_negative_2[validation_generator_2.classes[i]] +1

In [38]:
print('true_positive_2')
print(true_positive_2)
print('false_positive_2')
print(false_positive_2)
print('false_negative_2')
print(false_negative_2)
print('Total_2')
print(true_positive_2+false_negative_2)

true_positive_2
[  42.  130.]
false_positive_2
[ 5.  7.]
false_negative_2
[ 7.  5.]
Total_2
[  49.  135.]


In [39]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_generator = test_datagen.flow_from_directory(
                    'test',
                    shuffle=False,
                    target_size=(image_size,image_size),
                    batch_size=batch_size,
                    class_mode=None)


Found 794 images belonging to 1 classes.


In [40]:
predictions = model.predict_generator(test_generator,int(np.ceil(794/batch_size)))
class_ids = {train_generator.class_indices[x]: x for x in train_generator.class_indices}
predicted_classes = [class_ids[x] for x in np.argmax(predictions, axis=1)]


In [42]:
indexes = []
for i in range(len(np.argmax(predictions, axis=1))):
    if((np.argmax(predictions, axis=1)[i]==0) or (np.argmax(predictions, axis=1)[i]==6)):
        indexes.append(i)

In [43]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_generator = test_datagen.flow_from_directory(
                    'test',
                    shuffle=False,
                    target_size=(image_size_2,image_size_2),
                    batch_size=batch_size,
                    class_mode=None)


Found 794 images belonging to 1 classes.


In [44]:
predictions_2 = model_2.predict_generator(test_generator,int(np.ceil(794/batch_size)))
class_ids_2 = {train_generator_2.class_indices[x]: x for x in train_generator_2.class_indices}
predicted_classes_2 = [class_ids_2[x] for x in np.argmax(predictions_2, axis=1)]


In [52]:
for i in range(len(indexes)):
    predicted_classes[indexes[i]] = predicted_classes_2[indexes[i]]

In [53]:
num_class_0 = 0
num_class_6 = 0
for i in predicted_classes:
    if(i=='Black-grass'):
        num_class_0 = num_class_0 +1
    elif(i=='Loose Silky-bent'):
        num_class_6 = num_class_6 +1
        

In [54]:
from os.path import basename
test_ids = [basename(x) for x in test_generator.filenames]
submission = pd.DataFrame({'file':test_ids,'species':predicted_classes})
submission.to_csv('submission_mobilenet_two_models_v3_second_split.csv', encoding="utf8", index=False)

In [55]:
from IPython.display import FileLink
FileLink('submission_mobilenet_two_models_v3_second_split.csv')

/home/janardhan/plant_seeding/submission_mobilenet_two_models_v3_second_split.csv

In [ ]:
# 97.607 weights averaged